In [1]:
from contextlib import nullcontext

from math import floor

import h5py
import numpy as np
from PIL import Image
import numpy as np
from flatbuffers.packer import float64
from sympy import ceiling
from tensorflow.python.util.numpy_compat import np_array
#from CNN import X_train

2025-12-07 23:02:38.224345: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Gives the bbox and label for n-th box

In [2]:
#f = h5py.File("data/train_digitStruct.mat", "r")
#bbox_refs = f["digitStruct"]["bbox"]
#names_refs = f["digitStruct"]["name"]

def read_attr(attr):
    """Read a bbox attribute (left/top/width/height/label)."""
    # If it's a single reference
    if len(attr) == 1:
        return attr[0][0]
    # If multiple references
    return np.array([a[0] for a in attr]).astype(float)

def read_string(ref):
    return "".join([chr(c[0]) for c in f[ref]])

def get_name(i, name_refs):
    name_ref = name_refs[i-1][0]
    return read_string(name_ref)

def get_boxes(img_num, f, bbox_refs, name_refs, path):
    master_ref = bbox_refs[img_num-1][0]
    bbox_data_struct = f[master_ref]
    label_refs = bbox_data_struct["label"]
    num_of_digits = label_refs.shape[0]

    def get_field(field, digit_num):
        field_refs = bbox_data_struct[field]

        # if only 1 digit in img then refrence is to float
        if (field_refs.dtype == np.dtype("float64")):
            field_value = field_refs[0][0].astype("int16")
        else: # otherwise have to get the that the specific digit_num
            field_value = f[field_refs[digit_num][0]][0][0].astype("int16")

        return field_value

    img_data = []
    label_data = []
    filepath = path + get_name(img_num, name_refs)
    width, height = Image.open(filepath).size
    for i in range(num_of_digits):
        if height > width:
            x = (get_field("left", i) + (height - width) / 2)/ height
            y = get_field("top", i) / height
            box_height = get_field("height", i) / height
            box_width = get_field("width", i) / height
        else:
            x = get_field("left", i) / width
            y = (get_field("top", i) + (width - height) / 2) / width
            box_height = get_field("height", i) / width
            box_width = get_field("width", i) / width
        digit_arr = np.array([x, y, box_height, box_width], dtype='float32')
        img_data.append(digit_arr)
        label_data.append(get_field("label", i))
    digit_arr = np.array([0,0,0,0], dtype='float32')
    while len(img_data) < 6:
        img_data.append(digit_arr)
        label_data.append(0)
    return img_data, label_data

X_train

In [3]:
nrOfPictures_train = 33402
nrOfPictures_extra = 202353
total = nrOfPictures_train + nrOfPictures_extra

In [ ]:
from PIL import Image
import numpy as np
import tensorflow as tf
tf.config.set_visible_devices([], 'GPU')
def picturesToNPY(path, nrOfPictures, list, idx):
    for i in range(1,nrOfPictures+1):
        name = str(i) + ".png"
        image = Image.open(path + name)
        image = tf.image.rgb_to_grayscale(image)
        image = tf.image.resize_with_pad(image, 128, 128)
        image_array = np.array(image, dtype='float32') / 255
        list[idx] = image_array
        idx = idx + 1
        if (i % 1000 == 0):
            print(i)
    return idx



picture_data = np.empty(shape=(total,128,128,1), dtype='float32')
index = picturesToNPY("data/train/", nrOfPictures_train, picture_data, 0)
print("DONE-1")
index = picturesToNPY("data/extra/", nrOfPictures_extra, picture_data, index)
print("DONE-2")
np.savez_compressed("data/X_train", picture_data)
picture_data = None


In [4]:

def digitDataToNPY(file, nrOfPictures, box_list, label_list, idx, path):
    bbox_refs = file["digitStruct"]["bbox"]
    name_refs = file["digitStruct"]["name"]
    for i in range(1,nrOfPictures+1):
        bboxes, labels = get_boxes(i, file, bbox_refs, name_refs, path)
        box_list[idx] = bboxes
        label_list[idx] = labels
        idx = idx + 1
        if (i % 1000 == 0):
            print(i)
    return idx

bbox_array = np.empty(shape=(total, 6, 4))
labels_array = np.empty(shape=(total, 6))
f = h5py.File("data/train_digitStruct.mat", "r")
index = digitDataToNPY(f, nrOfPictures_train, bbox_array, labels_array, 0, 'data/train/')
f.close()
f = h5py.File("data/extra_digitStruct.mat", "r")
index = digitDataToNPY(f, nrOfPictures_extra, bbox_array, labels_array, index, 'data/extra/')

np.savez_compressed("data/bboxes_train", bbox_array)
np.savez_compressed("data/labels_train", labels_array)
#np.savez_compressed("data/labels_train_2", np.array(labels_array))



1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000
121000
122000
123000
124000
125000
126000
127000
128000
129000
130000
131000
13

In [5]:

bbox_array = None
labels_array = None
bbox_refs = None
f.close()

This displays images and the bounding boxes of the digits (1-500 checked), bad images in bad_data

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import numpy as np
from PIL import Image
import os
plt.close("all")

def visualize_fixed_bboxes(image_dir, y_train, num_show, rows, cols):
    """
    Loads images and displays them with matplotlib bounding boxes, handling
    NORMALIZED [Left, Top, Height, Width] format by swapping H and W for plotting.
    """

    image_files = sorted(
        [f for f in os.listdir(image_dir) if f.endswith(('.png', '.jpg', '.jpeg'))],
        key=lambda x: int(os.path.splitext(x)[0]) if os.path.splitext(x)[0].isdigit() else x
    )

    if not image_files:
        print(f"Error: No images found in {image_dir}")
        return

    num_show = min(num_show, len(image_files), len(y_train))

    # fig, axes = plt.subplots(1, num_show, figsize=(4 * num_show, 4))
    # if num_show == 1:
    #     axes = [axes]


    fig, axes = plt.subplots(rows, cols, figsize=(3 * cols, 3 * rows))
    axes = axes.flatten()

    for i in range(num_show):
        try:
            # 1. Load the Image and get dimensions
            img_path = os.path.join(image_dir, image_files[i])
            image = Image.open(img_path).convert('RGB')
            image_array = np.array(image)

            width, height = image.size
            max_length = max(width, height)

            ax = axes[i]
            ax.imshow(image_array)

            boxes_for_image = y_train[i]

            for bbox in boxes_for_image:
                # The format in y_train is: [L, T, H, W]
                normalized_left, normalized_top, normalized_height, normalized_width = bbox

                if np.sum(bbox) == 0:
                    continue

                # --- RESCALE COORDINATES ---
                left_pixel = normalized_left * max_length
                top_pixel = normalized_top * max_length

                # The crucial fix: When plotting, we use the value stored in the W index for width,
                # and the value stored in the H index for height.
                # W is at index 3, H is at index 2
                pixel_width = normalized_width * max_length
                pixel_height = normalized_height * max_length

                # 4. Create the Matplotlib Bounding Box Patch
                # Matplotlib requires (Left, Top, Width, Height)
                rect = patches.Rectangle(
                    (left_pixel, top_pixel), # Top-Left corner
                    pixel_width,             # Width (from normalized_width)
                    pixel_height,            # Height (from normalized_height)
                    linewidth=2,
                    edgecolor='blue',
                    facecolor='none'
                )

                ax.add_patch(rect)

            ax.set_title(f"Image: {image_files[i]}\n(L, T, H, W format)")
            ax.axis('off')

        except Exception as e:
            print(f"Could not process image {image_files[i]}: {e}")

    # Hide any unused subplots if num_show is less than ROWS * COLS
    for j in range(num_show, len(axes)):
        fig.delaxes(axes[j])

    plt.tight_layout()
    plt.show()

NUM_IMAGES_TO_SHOW = 500
IMAGE_DIR = 'data/train/'
COLS = 5
ROWS = int(np.ceil(NUM_IMAGES_TO_SHOW / COLS))
print(f"Attempting to display {NUM_IMAGES_TO_SHOW} images from {IMAGE_DIR} with normalized bboxes...")
visualize_fixed_bboxes(IMAGE_DIR, bbox_array, NUM_IMAGES_TO_SHOW, ROWS, COLS)
